In [114]:
import os
import math
import numpy as np
import pandas as pd
import datetime
import time
import random
from datetime import date
import pandas_ta as ta
from ta.volatility import BollingerBands
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from yahoo_fin import stock_info as si

In [115]:
#  Based on: https://tradingstrategyguides.com/best-combination-of-technical-indicators/

In [116]:
"""
def load_historic_data(symbol):
    #  Create output file name
    daily_file_name = "{}_daily.csv".format(symbol)
    daily_full_path = os.path.join('C:\\dev\\trading\\tradesystem1\\data\\daily_data', daily_file_name)

    #  Check if output file exists
    if os.path.exists(daily_full_path):
        df = pd.read_csv(daily_full_path, parse_dates=True)
        df['date'] = pd.to_datetime(df['date'])
        return df
    else:
        return None
    
df = load_historic_data('TSLA')
"""
def load_historic_data(symbol):
    today = datetime.date.today()
    today_str = today.strftime("%Y-%m-%d")
    # Download data from Yahoo Finance
    try:
        df = si.get_data(symbol, start_date=None, end_date=today_str, index_as_date=False)
        return df
    except:
        print('Error loading stock data for ' + symbol)
        return None
    
df = load_historic_data('TSLA')


#  Get the last year of data
df = df.tail(251)
df.reset_index(inplace=True)

In [117]:
def calculate_bollinger_bands(df):
    # Initialize Bollinger Bands Indicator
    indicator_bb = BollingerBands(close=df["adjclose"], window=20, window_dev=2)

    # Add Bollinger Bands features
    df['BB_mid'] = indicator_bb.bollinger_mavg()
    df['BB_high'] = indicator_bb.bollinger_hband()
    df['BB_low'] = indicator_bb.bollinger_lband()

    return df
    
    
df = calculate_bollinger_bands(df)

In [118]:
def calculate_rsi(df):
    df['RSI'] = ta.rsi(df['adjclose'], length=14)
    return df

df = calculate_rsi(df)

In [119]:
def calculate_ema(df):
    df["EMA_20"] = ta.ema(df["adjclose"], length=20)
    return df

df = calculate_ema(df)

In [120]:
def calculate_obv(df):
    df['OBV'] = ta.obv(df['adjclose'], df['volume'])
    return df

df = calculate_obv(df)

In [121]:
def calculate_strategy_rules(df):
    #  Entry Rule 1: Close price above Mid Bollinger Band
    df['BB_mid_ind'] = np.where(df["adjclose"] > df["BB_mid"], 1, 0)

    #  Entry Rule 2: RSI > 50
    df['RSI_ind'] = np.where(df["RSI"] > 50, 1, 0)

    #  Entry Rule 3: OBV is above the 20 EMA
    df['OBV_ind'] = np.where(df["OBV"] > df["EMA_20"], 1, 0)
    
    #  Stop Loss and exit rule
    df['stop_loss_ind'] = np.where(df["adjclose"] < df["BB_low"], 1, 0)
    return df

df = calculate_strategy_rules(df)

In [122]:
#  Assumption: Entry signals need to align in the last 10 trading days in order generate a buy signal
def execute_strategy(close_prices, bb_mid_inds, rsi_inds, obv_inds, stop_inds):
    lookback_period = 10
    entry_prices = []
    exit_prices = []
    entry_signal = 0
    exit_signal = 0
    
    for i in range(len(close_prices)):
        #  Evaluate entry strategy
        bb_signal = 0
        rsi_signal = 0
        obv_signal = 0
        check_entry_signals = 0
        #  Look for combined signals considering the last x trading days
        for j in range(lookback_period):
            lookback_ind = i - lookback_period + j
            if lookback_ind < 0:
                continue
            if bb_mid_inds[lookback_ind] == 1:
                bb_signal = 1
            if rsi_inds[lookback_ind] == 1:
                rsi_signal = 1      
            if obv_inds[lookback_ind] == 1:
                obv_signal = 1    
        #  All entry signals have to align
        if bb_signal == 1 and rsi_signal == 1 and obv_signal == 1:
            check_entry_signals = 1
            
        #  Add entry prices
        if entry_signal == 0 and check_entry_signals == 1:
            entry_prices.append(close_prices[i])
            exit_prices.append(np.nan)  
            entry_signal = 1
            exit_signal = 0
        #  Evaluate exit strategy
        elif entry_signal == 1 and stop_inds[i] == 1:
            entry_prices.append(np.nan)
            exit_prices.append(close_prices[i]) 
            entry_signal = 0
            exit_signal = 1
        else:
            #  Neither entry nor exit
            entry_prices.append(np.nan) 
            exit_prices.append(np.nan) 
            
    return entry_prices, exit_prices
    
entry_prices, exit_prices = execute_strategy(df['adjclose'],df['BB_mid_ind'], df['RSI_ind'], df['OBV_ind'], df['stop_loss_ind'])

In [123]:
def plot_graph(df, entry_prices, exit_prices, bb_high, bb_mid, bb_low, rsi, ema, obv):
    fig = make_subplots(rows=3, cols=1)

    #  Plot close price
    fig.add_trace(go.Line(x = df.index, y = df['adjclose'], line=dict(color="blue", width=1), name="Close"), row = 1, col = 1)
    
    #  Plot bollinger bands
    fig.add_trace(go.Line(x = df.index, y = bb_high, line=dict(color="#ffdf80", width=1), name="BB High"), row = 1, col = 1)
    fig.add_trace(go.Line(x = df.index, y = bb_mid, line=dict(color="#ffd866", width=1), name="BB Mid"), row = 1, col = 1)
    fig.add_trace(go.Line(x = df.index, y = bb_low, line=dict(color="#ffd24d", width=1), name="BB Low"), row = 1, col = 1)
    
    # Plot RSI
    fig.add_trace(go.Line(x = df.index, y = rsi, line=dict(color="#ffb299", width=1), name="RSI"), row = 2, col = 1)
    
    # Plot EMA
    fig.add_trace(go.Line(x = df.index, y = ema, line=dict(color="#99b3ff", width=1), name="EMA"), row = 1, col = 1)
    
    # Plot OBV
    fig.add_trace(go.Line(x = df.index, y = obv, line=dict(color="#99b3ff", width=1), name="OBV"), row = 3, col = 1)
    
    #  Add buy and sell indicators
    fig.add_trace(go.Scatter(x=df.index, y=entry_prices, marker_symbol="arrow-up", marker=dict(
        color='green',
    ),mode='markers',name='Buy'))
    fig.add_trace(go.Scatter(x=df.index, y=exit_prices, marker_symbol="arrow-down", marker=dict(
        color='red'
    ),mode='markers',name='Sell'))
    
    fig.update_layout(
        title={'text':'BB + RSI + OBV', 'x':0.5},
        autosize=False,
        width=900,height=1200)
    fig.update_yaxes(range=[0,1000000000],secondary_y=True)
    fig.update_yaxes(visible=False, secondary_y=True)  #hide range slider
    
    fig.show()
    
    
plot_graph(df, entry_prices, exit_prices, df['BB_high'], df['BB_mid'], df['BB_low'], df['RSI'], df['EMA_20'], df['OBV'])
    

C:\Users\dutch\anaconda3\envs\tradesystem1\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [124]:
def calculate_profit(entry_prices, exit_prices):
    entry_price = 0
    hold = 0
    profit = 0
    for i in range(len(entry_prices)):
        current_entry_price = entry_prices[i]
        current_exit_price = exit_prices[i]
        
        if not math.isnan(current_entry_price) and hold == 0:
            entry_price = current_entry_price
            hold = 1
        elif hold == 1 and not math.isnan(current_exit_price):
            hold = 0
            profit += current_exit_price - entry_price
        
    return profit        

#  Assuming investment of 1 share    
profit = calculate_profit(entry_prices, exit_prices)
print('profit: $',math.trunc(profit))

profit: $ 179


In [125]:
def perform_analysis(symbol, df):
    df = df.reset_index()
    df = calculate_bollinger_bands(df)
    df = calculate_rsi(df)
    df = calculate_bollinger_bands(df)
    df = calculate_ema(df)
    df = calculate_obv(df)
    df = calculate_strategy_rules(df)
    
    entry_prices, exit_prices = execute_strategy(df['adjclose'],df['BB_mid_ind'], df['RSI_ind'], df['OBV_ind'], df['stop_loss_ind'])
    profit = calculate_profit(entry_prices, exit_prices)
    return profit

In [126]:
# Backtesting using NASDAQ 100
nasdaq_100_df = pd.read_csv('https://raw.githubusercontent.com/justmobiledev/trade_data/main/nasdaq_100.csv')
nasdaq_100 = nasdaq_100_df['Symbol'].to_numpy()

In [127]:
total_profit = 0
for symbol in nasdaq_100:
    df = load_historic_data(symbol)
    df.reset_index(inplace=True)
    
    #  Random interval between remote fetch to avoid spam issues
    random_secs = random.uniform(0, 1)
    time.sleep(random_secs)
    
    #  Run backtest
    profit = perform_analysis(symbol, df)
    print(f"Backtest profit for symbol {symbol}: ${math.trunc(profit)}")
    total_profit += profit
  
print(f"\nAvg backtest profit per stock: ${math.trunc(total_profit / len(nasdaq_100))}")

Backtest profit for symbol AAPL: $131
Backtest profit for symbol ABNB: $-7
Backtest profit for symbol ADBE: $329
Backtest profit for symbol ADI: $87
Backtest profit for symbol ADP: $149
Backtest profit for symbol ADSK: $153
Backtest profit for symbol AEP: $48
Backtest profit for symbol ALGN: $364
Backtest profit for symbol AMAT: $69
Backtest profit for symbol AMD: $76
Avg backtest profit per stock: $140
